In [1]:
try:
    import google.colab
    inColab = True
except ImportError:
    inColab = False

if inColab == True:
    !pip install -U pandas==2.2.2 scipy==1.14.1 accelerate==1.6.0 peft==0.15.2 bitsandbytes==0.45.5 transformers==4.51.3 trl==0.16.1 datasets==3.5.0 tensorboard==2.19.0

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
    GenerationConfig
)
from transformers import AutoConfig,AutoModel
import torch
from peft import PeftModel, PeftConfig

if inColab == True:
    from google.colab import drive
    drive.mount("/content/gdrive")

In [2]:
import huggingface_hub
huggingface_hub.login()

In [3]:
    # "google/gemma-7b-it",
    # torch_dtype=torch.bfloat16,  # 또는 torch.float32
    # device_map="auto"

In [4]:
"""# 1. Load base and new model"""

## base 모델
# base_model =  "google/gemma-2b-it"
base_model = "google/gemma-3-4b-it"
## 파인튜닝한 모델 저장 위치 설정

# new_model = "/content/drive/MyDrive/Colab Notebooks/models/gemma_tune_2025_06_08_10"
new_model = "./models/gemma_tune_2025_07_18_16"
### 베이스모델 불러오기
baseModel = AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.bfloat16,
    device_map= "auto"
)

### 토크나이저 불러오기
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# PEFT 설정 로드
peft_config = PeftConfig.from_pretrained(new_model)
print(peft_config)

# 베이스모델에 어댑터 적용
model = PeftModel.from_pretrained(baseModel, new_model)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LoraConfig(task_type='CAUSAL_LM', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='google/gemma-3-4b-it', revision=None, inference_mode=True, r=64, target_modules={'down_proj', 'up_proj', 'gate_proj', 'o_proj', 'k_proj', 'q_proj', 'v_proj'}, exclude_modules=None, lora_alpha=16, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)


In [6]:
# 시스템 메시지 예시
DEFAULT_SYSTEM_MESSAGE = "당신은 전문의 수준의 의료 문제를 정확하게 답변하는 AI입니다."
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

In [9]:
# 시스템 메시지 + 문제(질문) 기반 답변 생성 함수

def generate_gemma_answer(model, tokenizer, user_message, system_message=None, device=DEVICE,
                          max_new_tokens=512, temperature=0.2, top_p=0.95, top_k=50):
    if system_message:
        prompt = (
            f"<start_of_turn>system\n{system_message}<end_of_turn>\n"
            f"<start_of_turn>user\n{user_message}<end_of_turn>\n"
            f"<start_of_turn>model\n"
        )
    else:
        prompt = (
            f"<start_of_turn>user\n{user_message}<end_of_turn>\n"
            f"<start_of_turn>model\n"
        )
    inputs = tokenizer(prompt, return_tensors='pt').to(device)
    model.to(device)
    model.eval()
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)




In [10]:
# 문제(질문) 예시
user_message = (
    "23세 여자가 3개월 전부터 기침을 한다며 내원했다. 기침은 밤에 누워 자려고 할 때 심해진다고 한다. "
    "1년 전에도 같은 시기에 기침이 3개월 동안 지속되다가 저절로 호전된 병력이 있다. 콧물이나 인후부 불편감은 없으며, "
    "비흡연자이고 복용 중인 약물도 없다. 신체검사에서 혈압 120/80mmHg, 맥박 78회/분, 호흡 18회/분, 체온 36.5°C로 측정되었다. "
    "청진상 호흡음은 정상이었고, 가슴 X선 사진과 코곁굴 X선 사진에서도 이상 소견이 없었다. 폐기능검사 결과는 다음과 같다.\n"
    "- 강제 폐활량(FVC): 정상 예측치의 91%\n"
    "- 1초간 강제날숨유량(FEV1): 정상 예측치의 85%\n"
    "- 1초간 강제날숨유량/강제폐활량(FEV1/FVC): 75%\n\n"
    "이 환자에서 다음으로 시행해야 할 검사는 무엇인가?\n"
    "1) 기관지내시경\n"
    "2) 기관지 확장제 반응 검사\n"
    "3) 가슴 컴퓨터단층촬영(CT)\n"
    "4) 메타콜린 기관지유발검사\n"
    "5) 폐 확산능 검사"
)
# 시스템 메시지(옵션)
system_message = DEFAULT_SYSTEM_MESSAGE
# 답변 생성
output = generate_gemma_answer(model, tokenizer, user_message, system_message)
print("\n[질문]\n", user_message)
print("\n[답변]\n", output)


[질문]
 23세 여자가 3개월 전부터 기침을 한다며 내원했다. 기침은 밤에 누워 자려고 할 때 심해진다고 한다. 1년 전에도 같은 시기에 기침이 3개월 동안 지속되다가 저절로 호전된 병력이 있다. 콧물이나 인후부 불편감은 없으며, 비흡연자이고 복용 중인 약물도 없다. 신체검사에서 혈압 120/80mmHg, 맥박 78회/분, 호흡 18회/분, 체온 36.5°C로 측정되었다. 청진상 호흡음은 정상이었고, 가슴 X선 사진과 코곁굴 X선 사진에서도 이상 소견이 없었다. 폐기능검사 결과는 다음과 같다.
- 강제 폐활량(FVC): 정상 예측치의 91%
- 1초간 강제날숨유량(FEV1): 정상 예측치의 85%
- 1초간 강제날숨유량/강제폐활량(FEV1/FVC): 75%

이 환자에서 다음으로 시행해야 할 검사는 무엇인가?
1) 기관지내시경
2) 기관지 확장제 반응 검사
3) 가슴 컴퓨터단층촬영(CT)
4) 메타콜린 기관지유발검사
5) 폐 확산능 검사

[답변]
 system
당신은 전문의 수준의 의료 문제를 정확하게 답변하는 AI입니다.
user
23세 여자가 3개월 전부터 기침을 한다며 내원했다. 기침은 밤에 누워 자려고 할 때 심해진다고 한다. 1년 전에도 같은 시기에 기침이 3개월 동안 지속되다가 저절로 호전된 병력이 있다. 콧물이나 인후부 불편감은 없으며, 비흡연자이고 복용 중인 약물도 없다. 신체검사에서 혈압 120/80mmHg, 맥박 78회/분, 호흡 18회/분, 체온 36.5°C로 측정되었다. 청진상 호흡음은 정상이었고, 가슴 X선 사진과 코곁굴 X선 사진에서도 이상 소견이 없었다. 폐기능검사 결과는 다음과 같다.
- 강제 폐활량(FVC): 정상 예측치의 91%
- 1초간 강제날숨유량(FEV1): 정상 예측치의 85%
- 1초간 강제날숨유량/강제폐활량(FEV1/FVC): 75%

이 환자에서 다음으로 시행해야 할 검사는 무엇인가?
1) 기관지내시경
2) 기관지 확장제 반응 검사
3) 가슴 컴퓨터단층촬영(CT)
4) 메타콜린 기관지유발검사
5) 폐 확산능 검

In [11]:
# 어댑터 병합
mergedModel = model.merge_and_unload()

# set your HF repository
hfAddr = "hyokwan/familidata_elysium"

# save model and tokenizer to HF hub
mergedModel.push_to_hub(hfAddr)
tokenizer.push_to_hub(hfAddr)

README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

C:\Users\HP\.finetune\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HP\.cache\huggingface\hub\models--hyokwan--familidata_elysium. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/hyokwan/familidata_elysium/commit/42aaad27cf770bd3527eab497534e284f8f5b7b2', commit_message='Upload tokenizer', commit_description='', oid='42aaad27cf770bd3527eab497534e284f8f5b7b2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/hyokwan/familidata_elysium', endpoint='https://huggingface.co', repo_type='model', repo_id='hyokwan/familidata_elysium'), pr_revision=None, pr_num=None)